In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib
import chart

In [ ]:
# matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rc('text', usetex=False)

In [ ]:
ppi = chart.Chart();

In [ ]:
a, r = np.mgrid[slice(0, 2.0 * np.pi,  np.pi / 180.0), slice(0, 60, 0.12)]
x = r * np.sin(a)
y = r * np.cos(a)
z = r * 1.0 - 15.0

In [ ]:
# ppi.set_data(x, y, z, extent=(-50,50,-50,50))
ppi.set_data(x, y, z)

In [ ]:
ppi.fig